# _Example notebook for AMR ancestry - Lázaro 2024_

## Description

This notebook provides an example of searching for variants in chromosome 1 for the AMR ancestry. 

It covers the following steps:

1. **Set up**
2. **Installing packages**
3. **Copy over data**
4. **Create a covariate file with GP2 data**
5. **Handle related individuals and other (non-PD) phenotypes**
6. **Case/Control Frequencies**
7. **Adjusted GLM model**

# Set up 

In [ ]:
# Use the os package to interact with the environment
import os
import sys

# Bring in Pandas for Dataframe functionality
import pandas as pd
from functools import reduce

# Bring some visualization functionality 
import seaborn as sns  

# numpy for basics
import numpy as np

# Use StringIO for working with file contents
from io import StringIO

# Enable IPython to display matplotlib graphs
import matplotlib.pyplot as plt
%matplotlib inline

# Enable interaction with the FireCloud API
from firecloud import api as fapi

# Import the iPython HTML rendering for displaying links to Google Cloud Console
from IPython.core.display import display, HTML

# Import urllib modules for building URLs to Google Cloud Console
import urllib.parse

# BigQuery for querying data
from google.cloud import bigquery

#Import Sys
import sys as sys

## Loading Python libraries and defining functions

In [ ]:
# Utility routine for printing a shell command before executing it
def shell_do(command):
    print(f'Executing: {command}', file=sys.stderr)
    !$command
    
def shell_return(command):
    print(f'Executing: {command}', file=sys.stderr)
    output = !$command
    return '\n'.join(output)

# Utility routine for printing a query before executing it
def bq_query(query):
    print(f'Executing: {query}', file=sys.stderr)
    return pd.read_gbq(query, project_id=BILLING_PROJECT_ID, dialect='standard')

# Utility routine for display a message and a link
def display_html_link(description, link_text, url):
    html = f'''
    <p>
    </p>
    <p>
    {description}
    <a target=_blank href="{url}">{link_text}</a>.
    </p>
    '''

    display(HTML(html))

# Utility routines for reading files from Google Cloud Storage
def gcs_read_file(path):
    """Return the contents of a file in GCS"""
    contents = !gsutil -u {BILLING_PROJECT_ID} cat {path}
    return '\n'.join(contents)
    
def gcs_read_csv(path, sep=None):
    """Return a DataFrame from the contents of a delimited file in GCS"""
    return pd.read_csv(StringIO(gcs_read_file(path)), sep=sep, engine='python')

# Utility routine for displaying a message and link to Cloud Console
def link_to_cloud_console_gcs(description, link_text, gcs_path):
    url = '{}?{}'.format(
        os.path.join('https://console.cloud.google.com/storage/browser',
                     gcs_path.replace("gs://","")),
        urllib.parse.urlencode({'userProject': BILLING_PROJECT_ID}))

    display_html_link(description, link_text, url)

## Set paths

In [ ]:
# Set up billing project and data path variables
BILLING_PROJECT_ID = os.environ['GOOGLE_PROJECT']
WORKSPACE_NAMESPACE = os.environ['WORKSPACE_NAMESPACE']
WORKSPACE_NAME = os.environ['WORKSPACE_NAME']
WORKSPACE_BUCKET = os.environ['WORKSPACE_BUCKET']
WORKSPACE_ATTRIBUTES = fapi.get_workspace(WORKSPACE_NAMESPACE, WORKSPACE_NAME).json().get('workspace',{}).get('attributes',{})

## Print the information to check we are in the proper release and billing 
## This will be different for you, the user, depending on the billing project your workspace is on
print('Billing and Workspace')
print(f'Workspace Name @ `WORKSPACE_NAME`: {WORKSPACE_NAME}')
print(f'Billing Project @ `BILLING_PROJECT_ID`: {BILLING_PROJECT_ID}')
print(f'Workspace Bucket, where you can upload and download data @ `WORKSPACE_BUCKET`: {WORKSPACE_BUCKET}')
print('')

## AMP-PD v3.0
## Explicitly define release v3.0 path 
AMP_RELEASE_PATH = 'gs://amp-pd-data/releases/release/path'  ##  Enter valid release path
AMP_CLINICAL_RELEASE_PATH = f'{AMP_RELEASE_PATH}/clinical'
AMP_RELEASE_GATK_PATH = os.path.join(AMP_RELEASE_PATH, 'gatk')
AMP_WGS_RELEASE_PATH = 'gs://amp-pd-genomics/releases/release/path'   ##  Enter valid release path
AMP_WGS_RELEASE_PLINK_PATH = os.path.join(AMP_WGS_RELEASE_PATH, 'plink')
AMP_WGS_RELEASE_PLINK_PFILES = os.path.join(AMP_WGS_RELEASE_PLINK_PATH, 'pfiles')

print('AMP-PD v3.0')
print(f'Path to AMP-PD v3.0 Clinical Data: {AMP_CLINICAL_RELEASE_PATH}')
print(f'Path to AMP-PD v3.0 WGS Data: {AMP_WGS_RELEASE_PLINK_PATH}')
print(f'Path to AMP-PD v3.0 WGS Data: {AMP_WGS_RELEASE_PLINK_PFILES}')
print('')

## GP2 v7.0
## Explicitly define release v7.0 path 
GP2_RELEASE_PATH = '/GP2/release/path' ##  Enter valid GP2 Release path
GP2_CLINICAL_RELEASE_PATH = f'{GP2_RELEASE_PATH}/clinical_data'
GP2_RAW_GENO_PATH = f'{GP2_RELEASE_PATH}/raw_genotypes'
GP2_IMPUTED_GENO_PATH = f'{GP2_RELEASE_PATH}/imputed_genotypes'
GP2_META_RELEASE_PATH = f'{GP2_RELEASE_PATH}/meta_data'
GP2_SUMSTAT_RELEASE_PATH = f'{GP2_RELEASE_PATH}/summary_statistics'

print('GP2 v7.0')
print(f'Path to GP2 v7.0 Clinical Data @ `GP2_CLINICAL_RELEASE_PATH`: {GP2_CLINICAL_RELEASE_PATH}')
print(f'Path to GP2 v7.0 Metadata @ `GP2_META_RELEASE_PATH`: {GP2_META_RELEASE_PATH}')
print(f'Path to GP2 v7.0 Raw Genotype Data @ `GP2_RAW_GENO_PATH`: {GP2_RAW_GENO_PATH}')
print(f'Path to GP2 v7.0 Imputed Genotype Data @ `GP2_IMPUTED_GENO_PATH`: {GP2_IMPUTED_GENO_PATH}')
print(f'Path to GP2 v7.0 summary statistics: {GP2_SUMSTAT_RELEASE_PATH}')

## Make working directory

In [ ]:
ancestry = 'AMR'
WORK_DIR = f'{ancestry}'

! mkdir {WORK_DIR}

# Install Packages

## PLINK

In [ ]:
%%bash

mkdir -p ~/tools
cd ~/tools

if test -e /home/jupyter/tools/plink; then
echo "Plink1.9 is already installed in /home/jupyter/tools/"

else
echo -e "Downloading plink \n    -------"
wget -N http://s3.amazonaws.com/plink1-assets/plink_linux_x86_64_20190304.zip 
unzip -o plink_linux_x86_64_20190304.zip
echo -e "\n plink downloaded and unzipped in /home/jupyter/tools \n "

fi

In [ ]:
%%bash

mkdir -p ~/tools
cd ~/tools

if test -e /home/jupyter/tools/plink2; then
echo "Plink2 is already installed in /home/jupyter/tools/"

else
echo -e "Downloading plink2 \n    -------"
wget -N https://s3.amazonaws.com/plink2-assets/alpha5/plink2_linux_amd_avx2_20240625.zip
unzip -o plink2_linux_amd_avx2_20240625.zip
echo -e "\n plink2 downloaded and unzipped in /home/jupyter/tools \n "

fi

# Copy Over Files 

In [ ]:
##  Imputed genotype GP2 Tier 2 data
## In this case, we will use tyhe chromosome 1 as an example 


shell_do(f'gsutil -u {BILLING_PROJECT_ID} -m cp -r {GP2_IMPUTED_GENO_PATH}/AMR/chr1_AMR_release7* {WORK_DIR}')


In [ ]:
## clinical data 

## master key
shell_do(f'gsutil -u {BILLING_PROJECT_ID} -m cp {GP2_CLINICAL_RELEASE_PATH}/master_key_release7_final.csv {WORK_DIR}')

## related file 
shell_do(f'gsutil -u {BILLING_PROJECT_ID} -m cp {GP2_META_RELEASE_PATH}/related_samples/{ancestry}_release7.related {WORK_DIR}')


In [ ]:
## PCs
shell_do(f'gsutil -u {BILLING_PROJECT_ID} -m cp {GP2_RAW_GENO_PATH}/{ancestry}/{ancestry}_release7.eigenvec {WORK_DIR}')


In [ ]:
## Check files
! ls {WORK_DIR}

# Create a covariate file with GP2 data

Covariate file is creating by putting together clinical, PC and genetic files 

In [ ]:
# Clinical master key
clin = pd.read_csv(f'{WORK_DIR}/master_key_release7_final.csv')
clin.info()

In [ ]:
#Genetic data
gen = pd.read_csv(f'{WORK_DIR}/chr1_AMR_release7.psam', sep='\t')
gen.info()

In [ ]:
pcs = pd.read_csv(f'{WORK_DIR}/AMR_release7.eigenvec', sep='\t')
pcs.info()

In [ ]:
gen2 = pd.merge(gen, clin, left_on='#IID', right_on='GP2sampleID')
gen2.info()

In [ ]:
gen3 = pd.merge(gen2, pcs, left_on='#IID', right_on='IID')
gen3.info()

In [ ]:
# Subsetting to keep only a few columns 
plink_clin = gen3[['#IID', 'SEX', 'PHENO1', 'age_at_sample_collection', 'PC1', 'PC2', 'PC3', 'PC4', 'PC5']]
plink_clin.info()

In [ ]:
#Rename age_at_sample_collection  
plink_clin = plink_clin.rename(columns={'age_at_sample_collection': 'AGE'})


In [ ]:
plink_clin.to_csv(f'{WORK_DIR}/{ancestry}covars.txt', sep='\t', index=False, na_rep='-9',)

In [ ]:
covars = pd.read_csv(f'{WORK_DIR}/{ancestry}covars.txt', sep='\t')
covars.info()

# Handle related individuals and other (non-PD) phenotypes

## Related samples

In [ ]:
# Here you can load the related file 
related_df = pd.read_csv(f'{WORK_DIR}/{ancestry}_release7.related')
print(related_df.shape)


In [ ]:
#We create a list of individuals to remove from the dataframe
! cut -d, -f2 {WORK_DIR}/AMR_release7.related > {WORK_DIR}/related_ids.txt

In [ ]:
## Here we remove them from the genetic file

In [ ]:
!/home/jupyter/tools/plink2 \
--pfile {WORK_DIR}/chr1_AMR_release7 \
--remove {WORK_DIR}/related_ids.txt \
--make-pgen \
--out {WORK_DIR}/chr1_AMR_release7_nonrelated

## Non-PD phenotypes

In [ ]:
WORK_DIR = f'{ancestry}'

! /home/jupyter/tools/plink2 \
--pfile {WORK_DIR}/chr1_AMR_release7_nonrelated  \
--prune \
--make-pgen \
--out {WORK_DIR}/chr1_AMR_release7_nonrelated_pdc

# Case/Control Frequencies

##### Glossary

- CHR	Chromosome code
- SNP	Variant identifier
- A1	Allele 1 (usually minor)
- A2	Allele 2 (usually major)
- MAF	Allele 1 frequency in all subjects
- F_A/MAF_A	Allele 1 frequency in cases
- F_U/MAF_U	Allele 1 frequency in controls
- NCHROBS_A	Number of case allele observations
- NCHROBS_U	Number of control allele observations

## Extract the region of interest

In this section, you can create a file with the format CHR BP_START BP_END of the SNPs you are interested in studying. As an example, we create in the next chunk a file "range_chr1.txt" with 2 SNPs in chromosome 1

In [ ]:
SNP_coordinates = [
    "1 226728377 226728377",
    "1 11796321 11796321"
]

# Write to a text file
with open("AMR/range_chr1.txt", "w") as file:
    file.write("\n".join(SNP_coordinates))

In [ ]:
# Extract the regions and create plink and plink2 files

In [ ]:
! ls {WORK_DIR}

In [ ]:
WORK_DIR = f'{ancestry}'

! /home/jupyter/tools/plink2 \
--pfile {WORK_DIR}/chr1_AMR_release7_nonrelated_pdc \
--extract range {WORK_DIR}/range_chr1.txt \
--make-pgen \
--out {WORK_DIR}/snpextracted

In [ ]:
WORK_DIR = f'{ancestry}'

! /home/jupyter/tools/plink2 \
--pfile {WORK_DIR}/snpextracted \
--make-bed \
--out {WORK_DIR}/snpextracted_plink1

## Calculate frequencies using Plink 1

In [ ]:
WORK_DIR = f'{ancestry}'

! /home/jupyter/tools/plink \
--bfile {WORK_DIR}/snpextracted_plink1 \
--assoc \
--ci 0.95 \
--out {WORK_DIR}/AMR_assoc


In [ ]:
! cat {WORK_DIR}/AMR_assoc.assoc

# Calculate glm (adjusted model) using Plink 2

In [ ]:
WORK_DIR = f'{ancestry}'

! /home/jupyter/tools/plink2 \
--pfile {WORK_DIR}/snpextracted \
--glm hide-covar firth-fallback pheno-ids \
--covar-name AGE,SEX,PC1,PC2,PC3,PC4,PC5 \
--pheno-name PHENO1 \
--pheno {WORK_DIR}/{ancestry}covars.txt \
--ci 0.95 \
--covar-variance-standardize \
--covar {WORK_DIR}/{ancestry}covars.txt \
--out {WORK_DIR}/AMR_glm

In [ ]:
! cat {WORK_DIR}/AMR_glm.PHENO1.glm.logistic.hybrid